In [23]:
import numpy as np
from scipy.signal import chirp
from scipy.io.wavfile import write

In [ ]:
# Parameters
T_sec   = 3.0     # Duration of the chirp (in seconds)
T_start = 0.1     # Duration of silence at the start (in seconds)
T_end   = 0.1     # Duration of silence at the end (in seconds)
fs      = 44100   # Sampling rate (Hz)
f_start = 50      # Start frequency of the chirp (Hz)
f_end   = 1_000    # End frequency of the chirp (Hz)

In [25]:
def generate_chirp(T_sec, T_start, T_end, fs, f_start, f_end):
    # Time array for the chirp
    t = np.linspace(0, T_sec, int(T_sec * fs), endpoint=False)

    # Generate the logarithmic chirp signal
    chirp_signal = chirp(t, f0=f_start, t1=T_sec, f1=f_end, method='logarithmic')

    # Generate silence arrays for the start and end
    silence_start = np.zeros(int(T_start * fs))
    silence_end   = np.zeros(int(T_end * fs))

    # Concatenate silence at the beginning, the chirp, and silence at the end
    final_signal = np.concatenate([silence_start, chirp_signal, silence_end])
    return final_signal

In [26]:
final_signal = generate_chirp(T_sec, T_start, T_end, fs, f_start, f_end)

In [27]:
def save_signal_as_wav(filename, signal, sampling_rate):
    if np.issubdtype(signal.dtype, np.floating):
        signal = np.int16(signal / np.max(np.abs(signal)) * 32767)

    # Write to WAV file
    write(filename, sampling_rate, signal)
    print(f"Audio file saved as: {filename}")

In [ ]:
save_signal_as_wav(f'../data/audio_samples/chirp_{f_start}_{f_end}_{T_sec}sec.wav', final_signal, fs)

In [34]:
def freq_to_time(f_target, fs, f_start=50, f_end=2_000, T_sec=0.1, T_start=0.1):
    # 1. Calculate the time within the chirp itself
    # Using the natural log (np.log)
    t_chirp = T_sec * (np.log(f_target / f_start) / np.log(f_end / f_start))

    # 2. Add the initial silence offset
    total_time = t_chirp + T_start

    # 3. Convert time to sample index
    sample_index = int(total_time * fs)

    return total_time, sample_index

In [30]:
target_f = 890 # feel free to change this

In [35]:
pred_time, pred_idx = freq_to_time(target_f, fs, f_start, f_end, T_sec, T_start)

# Verification: Look at the signal around the predicted index
# We measure the distance between zero-crossings to find the local frequency
window_size = 500
start_search = pred_idx - window_size
end_search = pred_idx + window_size
signal_slice = final_signal[start_search:end_search]

# Count zero crossings in the slice to calculate frequency
zero_crossings = np.where(np.diff(np.sign(signal_slice)))[0]
# Frequency = (Number of crossings / 2) / (Time duration of slice)
measured_f = (len(zero_crossings) / 2) / (len(signal_slice) / fs)

print(f"--- Results for {target_f}Hz ---")
print(f"Predicted Time:  {pred_time:.4f} s")
print(f"Predicted Index: {pred_idx}")
print(f"Measured Freq:  {measured_f:.2f} Hz")

# Check accuracy (allowing for small windowing error)
error = abs(measured_f - target_f)
if error < (target_f * 0.02): # Within 2% tolerance
    print("\n✅ TEST PASSED: The predicted index correctly identifies the frequency.")
else:
    print(f"\n❌ TEST FAILED: Error of {error:.2f} Hz is too high.")

--- Results for 890Hz ---
Predicted Time:  4.9055 s
Predicted Index: 216332
Measured Freq:  882.00 Hz

✅ TEST PASSED: The predicted index correctly identifies the frequency.
